<a href="https://colab.research.google.com/github/DivyaGaddam27/CCMLASSIGN3/blob/main/Insurance_claim_prediction_system_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Insurance_Train.csv to Insurance_Train.csv
Saving Insurance_Test.csv to Insurance_Test.csv


In [ ]:
# Load and explore the data
train_data = pd.read_csv('Insurance_Train.csv')
test_data = pd.read_csv('Insurance_Test.csv')

In [ ]:
train_data.head()

,id,age,job,marital,education,balance,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,y
0,98749,32,admin.,single,secondary,64,yes,no,unknown,may,202,2,-1,0,unknown,0
1,19550,45,blue-collar,married,secondary,534,no,no,cellular,aug,104,6,-1,0,unknown,0
2,75084,45,technician,married,secondary,1477,yes,no,cellular,nov,75,1,132,1,failure,0
3,65715,39,technician,married,tertiary,14,no,no,cellular,jan,114,2,-1,0,unknown,0
4,41412,49,blue-collar,single,unknown,2222,no,no,unknown,jun,114,2,-1,0,unknown,0


In [ ]:

categorical_columns = ['Job', 'Marital', 'Education', 'Housing', 'Loan', 'Contact', 'month', 'Poutcome']

train_data['balance_age_ratio'] = train_data['balance'] / train_data['age']
test_data['balance_age_ratio'] = test_data['balance'] / test_data['age']

train_data['duration_campaign_ratio'] = train_data['duration'] / train_data['campaign']
test_data['duration_campaign_ratio'] = test_data['duration'] / test_data['campaign']

train_data['is_retired'] = (train_data['job'] == 'retired').astype(int)
test_data['is_retired'] = (test_data['job'] == 'retired').astype(int)

train_data['previous_campaigns_success_rate'] = train_data['previous'] / train_data['campaign']
test_data['previous_campaigns_success_rate'] = test_data['previous'] / test_data['campaign']

train_data['contacted_before'] = (train_data['pdays'] > -1).astype(int)
test_data['contacted_before'] = (test_data['pdays'] > -1).astype(int)



In [ ]:
train_data = train_data.drop(train_data.columns[0], axis=1)
test_ids = test_data['id']
test_data = test_data.drop(test_data.columns[0], axis=1)
encoder = LabelEncoder()

for col in ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'poutcome']:
    train_data[col] = encoder.fit_transform(train_data[col])
    test_data[col] = encoder.transform(test_data[col])

In [ ]:
X = train_data.drop(['y'], axis=1)
y = train_data['y']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_data)
print(X_train_scaled)

[[ 3.77567019e-01 -1.33332367e+00 -2.73800766e-01 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]
 [-1.41097241e+00 -1.33332367e+00  1.37482830e+00 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]
 [-5.63769524e-01  1.41946125e+00 -2.73800766e-01 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]
 ...
 [ 1.03240191e-03 -1.02745868e+00 -2.73800766e-01 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]
 [-1.03443780e+00  5.01866276e-01  1.37482830e+00 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]
 [-1.59923972e+00 -1.02745868e+00 -2.73800766e-01 ... -2.34377026e-01
  -2.57853653e-01 -4.76247724e-01]]


In [ ]:
clf = RandomForestClassifier( max_depth=5, random_state=0, class_weight='balanced')
clf.fit(X_train_scaled, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=5, random_state=0)

In [ ]:
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
forest = RandomForestClassifier(max_depth=5, random_state=0, class_weight='balanced_subsample')
forest.fit(X_train_scaled, y_train)
y_pred_1 = forest.predict(X_test_scaled)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC

rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_resample(X_train_scaled, y_train)

svm = SVC(kernel='linear')
svm.fit(X_resampled, y_resampled)
svm_predictions=svm.predict(X_test_scaled)
forest = RandomForestClassifier(max_depth=1, random_state=0, class_weight='balanced')
forest.fit(X_resampled, y_resampled)
y_pred_1 = forest.predict(X_test_scaled)

In [ ]:
from sklearn.utils import class_weight

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(loss='log_loss',learning_rate=0.1,criterion='squared_error')
gb.fit(X_train_scaled, y_train)
gb_predictions=gb.predict(X_test_scaled)
y_val_pred = clf.predict(X_val_scaled)

In [ ]:
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy: {accuracy}")

Validation accuracy: 0.8006881297616122


In [ ]:
y_test_pred = clf.predict(X_test_scaled)
print(y_test_pred)

[1 0 0 ... 1 0 1]


In [ ]:
submission = pd.DataFrame({'id': test_ids, 'y': y_test_pred})
submission.to_csv('submission.csv', index=False)